In [8]:
def parseBoundingBoxesSingleType(filename):
    with open(filename) as f:
        contents = {}
        for line in f:
            line = line[:-1].split(',')
            if line[0] not in contents:
                contents[line[0]] = []
            bb = {}
            bb['xmin'] = float(line[2])
            bb['ymin'] = float(line[3])
            bb['xmax'] = float(line[4])
            bb['ymax'] = float(line[5])
            contents[line[0]].append(bb)
    return contents

In [9]:
#Function to calculate the average precision and recall
#Argument - list (elements are tuples(numerator and denominator))
#Returns - sum of numerators/sum of denominators
def computeAverage(listname):
    total = 0
    total2 = 0
    for i in range(len(listname)):
        total += listname[i][0]
        total2 += listname[i][1]
    return round(total/total2, 4)

In [10]:
#Function to calculate Recall
#Arguments - 2 dictionaries (Wei Li's parsed bounding boxes, Model's bounding boxes)
#Returns - a tuple (no.of boxes labelled correctly, no. of bounding boxes in ground truth labelling)
def calculateRecallSingleType(originalList, generatedList):
    
    #initialise variables
    labelCount = 0    
    denominator = len(originalList)
    buffer = 20
    
    #iterate through labelled bounding boxes
    for iterate in range(0, len(originalList)):
        for iterate2 in range(0, len(generatedList)):
            #check if model's bounding box coordinates are withing 20 pixels of corresponidng ground truth coordinates
            if(generatedList[iterate2]['ymin'] - buffer <= originalList[iterate]['ymin'] <= generatedList[iterate2]['ymin'] + buffer and
               generatedList[iterate2]['xmin'] - buffer <= originalList[iterate]['xmin'] <= generatedList[iterate2]['xmin'] + buffer and
               generatedList[iterate2]['ymax'] - buffer <= originalList[iterate]['ymax'] <= generatedList[iterate2]['ymax'] + buffer and
               generatedList[iterate2]['xmax'] - buffer <= originalList[iterate]['xmax'] <= generatedList[iterate2]['xmax'] + buffer ):
                labelCount += 1
                #if match found, then break
                break
        
                   
    return labelCount,denominator

In [11]:
#Function to calculate precision
#Arguments - 2 dictionaries (Wei Li's parsed bounding boxes, Model's bounding boxes)
#Returns - a tuple (no.of boxes labelled correctly, no. of bounding boxes model'slabelling)
def calculatePrecisionSingleType(originalList, generatedList):
    
    #initialise variables
    labelCount = 0    
    denominator = len(generatedList)
    buffer = 20
    
    #iterate through labelled bounding boxes
    for iterate in range(0, len(originalList)):
        for iterate2 in range(0, len(generatedList)):
            #check if model's bounding box coordinates are withing 20 pixels of corresponidng ground truth coordinates
            if(generatedList[iterate2]['ymin'] - buffer <= originalList[iterate]['ymin'] <= generatedList[iterate2]['ymin'] + buffer and
               generatedList[iterate2]['xmin'] - buffer <= originalList[iterate]['xmin'] <= generatedList[iterate2]['xmin'] + buffer and
               generatedList[iterate2]['ymax'] - buffer <= originalList[iterate]['ymax'] <= generatedList[iterate2]['ymax'] + buffer and
               generatedList[iterate2]['xmax'] - buffer <= originalList[iterate]['xmax'] <= generatedList[iterate2]['xmax'] + buffer ):
                labelCount += 1
                #if match found, then break
                break
        
    
    return labelCount,denominator

In [12]:
def IOUcal(dictlist1, dictlist2, IOU):
    
    for i in range(len(dictlist1)):
        for j in range(len(dictlist2)):
            
            Xi1 = dictlist1[i][1]
            Yi1 = dictlist1[i][0]
            Xj1 = dictlist2[j][1]
            Yj1 = dictlist2[j][0]
            
            Xi2 = dictlist1[i][3]
            Yi2 = dictlist1[i][2]
            Xj2 = dictlist2[j][3]
            Yj2 = dictlist2[j][2]

            xA = max(Xi1, Xj1)
            yA = max(Yi1, Yj1)
            xB = min(Xi2, Xj2)
            yB = min(Yi2, Yj2)
            
            
            if((Xi1 <= xA <= Xi2 and Xj1 <= xA <= Xj2) and (Yi1 <= yA <= Yi2 and Yj1 <= yA <= Yj2 ) and 
              (Xi1 <= xB <= Xi2 and Xj1 <= xB <= Xj2) and (Yi1 <= yB <= Yi2 and Yj1 <= yB <= Yj2 )):
    
                # compute the area of intersection rectangle
                interArea = (xB - xA ) * (yB - yA )

                # compute the area of both the prediction and ground-truth rectangles
                boxAArea = (dictlist1[i][3] - dictlist1[i][1]) * (dictlist1[i][2] - dictlist1[i][0])
                boxBArea = (dictlist2[j][3] - dictlist2[j][1]) * (dictlist2[j][2] - dictlist2[j][0])

                # compute the intersection over union by taking the intersection
                # area and dividing it by the sum of prediction + ground-truth
                # areas - the interesection area
                result = round(interArea/(boxAArea + boxBArea - interArea),4)
                
                if(result > 0.5): 
                    IOU.append(result)
                    break

    return IOU

In [13]:
#Generate precision and recall file
gt_bb = parseBoundingBoxesSingleType('new testlabels copy.csv')

#with open('../Epochs/300epochs/BoundingBoxes/results.csv', 'w+') as output:
    #output.write('Threshold,Precision,Recall,F1\n')
for n in [20]:
    output_bb = parseBoundingBoxesSingleType('predictions_0' + str(n) + 'thresh_inv.csv')
    print(gt_bb)
    print(output_bb)
    recall = []
    precision = []
    for image in gt_bb.keys():
        if image in output_bb.keys():
            recall.append(calculateRecallSingleType(gt_bb[image], output_bb[image]))
            precision.append(calculatePrecisionSingleType(gt_bb[image], output_bb[image]))
        else:
            print(image)
    print(precision)
    print(recall)
    avgRecall = computeAverage(recall)
    avgPrecision = computeAverage(precision)
    f1 = 2*avgPrecision*avgRecall/(avgPrecision + avgRecall)
    print('Threshold: 0.' + str(n))
    print('Recall: ' + str(avgRecall))
    print('Precision: ' + str(avgPrecision))
    print('F1: ' + str(f1))
    print('\n')
        #output.write(str(n/100)+','+str(avgPrecision)+','+str(avgRecall)+','+str(f1)+'\n')
    


{'1ROI_100kx_4100CL_foil1.jpg': [{'xmin': 104.0, 'ymin': 43.0, 'xmax': 198.0, 'ymax': 128.0}, {'xmin': 303.0, 'ymin': 164.0, 'xmax': 337.0, 'ymax': 213.0}, {'xmin': 301.0, 'ymin': 245.0, 'xmax': 331.0, 'ymax': 266.0}, {'xmin': 157.0, 'ymin': 156.0, 'xmax': 184.0, 'ymax': 182.0}, {'xmin': 119.0, 'ymin': 233.0, 'xmax': 144.0, 'ymax': 254.0}, {'xmin': 144.0, 'ymin': 289.0, 'xmax': 186.0, 'ymax': 327.0}, {'xmin': 205.0, 'ymin': 409.0, 'xmax': 236.0, 'ymax': 434.0}, {'xmin': 321.0, 'ymin': 467.0, 'xmax': 357.0, 'ymax': 490.0}, {'xmin': 137.0, 'ymin': 497.0, 'xmax': 202.0, 'ymax': 563.0}, {'xmin': 400.0, 'ymin': 501.0, 'xmax': 423.0, 'ymax': 534.0}, {'xmin': 360.0, 'ymin': 573.0, 'xmax': 393.0, 'ymax': 617.0}, {'xmin': 514.0, 'ymin': 616.0, 'xmax': 574.0, 'ymax': 697.0}, {'xmin': 580.0, 'ymin': 804.0, 'xmax': 633.0, 'ymax': 875.0}, {'xmin': 703.0, 'ymin': 789.0, 'xmax': 760.0, 'ymax': 871.0}, {'xmin': 756.0, 'ymin': 885.0, 'xmax': 795.0, 'ymax': 940.0}, {'xmin': 231.0, 'ymin': 729.0, 'xmax':

In [27]:
#Generate count file
gt_bb = parseBoundingBoxesSingleType('testlabels.csv')

with open('../BBcount.csv', 'w+') as output:
    output.write('Image,Correctly Labelled,Correctly Labelled,Total,Total_GT\n')
    for n in [20]:
        output_bb = parseBoundingBoxesSingleType('../BatchSize/16batchSize/BoundingBoxes/predictions_0' + str(n) + 'thresh_inv.csv')
        recall = []
        precision = []
        for image in gt_bb.keys():
            if image in output_bb.keys():
                count_gt = calculateRecallSingleType(gt_bb[image], output_bb[image])
                count_image = calculatePrecisionSingleType(gt_bb[image], output_bb[image])
                print(count_gt)
                print(count_image)
                recall.append(count_gt)
                precision.append(count_image)
                output.write(image+','+str(count_gt[0])+','+str(count_image[0])+','+str(count_image[1])+','+str(count_gt[1])+'\n')
        print(precision)
        print(recall)
        avgRecall = computeAverage(recall)
        avgPrecision = computeAverage(precision)
        f1 = 2*avgPrecision*avgRecall/(avgPrecision + avgRecall)
        print('Threshold: 0.' + str(n))
        print('Recall: ' + str(avgRecall))
        print('Precision: ' + str(avgPrecision))
        print('F1: ' + str(f1))
        print('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'testlabels.csv'